# 1. 命名实体识别详解
## 中文命名实体识别特点  
NER通常包括两部分：（1）实体边界识别；（2）确定实体类别（人名、地名、机构名或其他）。  
英文中的命名实体具有比较明显的形式标志（即实体中的每个词的第一个字母要大写），所以实体边界识别相对容易，任务的重点是确定实体的类别。和英文相比，中文命名实体识别任务更加复杂，而且相对于实体类别标注子任务，实体边界的识别更加困难。  

<img src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/161443041823784011614430417975.png"/>

## 实体标注体系（Entity labeling system）
大部分情况下，标签体系越复杂准确度也越高，但相应的训练时间也会增加。因此需要根据实际情况选择合适的标签体系。

|Tokens|IO|BIO|BIOES|
|---|---|---|---|
|突|O|O|O|
|发|O|O|O|
|心|I-SYM|B-SYM|B-SYM|
|绞|I-SYM|I-SYM|I-SYM|
|痛|I-SYM|I-SYM|E-SYM|
|，|O|O|O|
|未|I-NEG|B-NEG|S-NEG|
|加|I-DEV|B-DEV|B-DEV|
|重|I-DEV|I-DEV|E-DEV|
|。|O|O|O|

## 序列标注（Sequence Labeling）

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594735872317-8d16d2d9-ebc4-4ad7-a1f9-4a622ee97c75.png"/>

序列标注输入为特征序列，输出为类别序列。

## 分类的评价标准（Measure of classification）
Accuracy：准确率 = 预测正确 / 总样本
Precision：精确率 = 正确预测为正 / 预测为正
Recall：召回率 = 正确预测为正 / 真实为正
F1 score：1：1调和准确和召回

[如何辨别机器学习模型的好坏？秒懂Confusion Matrix](https://www.ycc.idv.tw/confusion-matrix.html)

<img src="https://cdn.nlark.com/yuque/0/2020/jpeg/1508544/1594736539902-c517c1b1-1cd2-4a11-a20c-09ffaa558834.jpeg"/>

## NER的评价标准
1. 基于token标签进行直接评测
2. 考虑实体边界 + 实体类型的评测  
    a. 完全匹配  
    b. 部分匹配（重叠）

- Message Understanding Conference（MUC）
    - Correct (COR): 匹配成功；
    - Incorrect (INC): 匹配失败；
    - Partial (PAR): 预测的实体边界与测试集重叠，但不完全相同；
    - Missing (MIS): 测试集实体边界没有被预测识别出来；
    - Spurius (SPU): 预测出的实体边界在测试集中不存在
    
1. 测试集标签个数统计（golden）：$Possible(POS)=TP+FN=COR+INC+PAR+MIS$ 
2. 预测结果标签个数统计（predict）: $Actual(ACT)=TP+FP=COR+INC+PAR+SPU$   
3. 精确匹配（exact）：$Precision=\frac{TP}{TP+FP}=\frac{COR}{ACT}$ $Recall=\frac{TP}{TP+FN}=\frac{COR}{POS}$        
4. 部分匹配（partial）：$Precision=\frac{TP}{TP+FP}=\frac{COR+0.5\times PAR}{ACT}$ $Recall=\frac{TP}{TP+FN}=\frac{COR+0.5\times PAR}{POS}$

## 多分类的评价标准
- Macro F1：将n分类的评价拆成n个二分类的评价，计算每个二分类的F1 score，n个F1 score的平均值即为Macro F1。
- Micro F1：将n分类的评价拆成n个二分类的评价，将n个二分类评价的TP、FP、RN对应相加，计算评价准确率和召回率，由这2个准确率和召回率计算的F1 score即为Micro F1。
- 注意：Macro F1受样本数量少的类别影响大。

# 2 HMM与维特比解码
## 隐马尔可夫模型实例

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594786428600-44fce29b-19f5-4ecc-a87c-398184943805.png"/>

## 隐马尔可夫模型（Hidden Markov Model）
- 马尔可夫过程为状态空间中经过从一个状态到另一个状态的转换的随机过程。
- 该过程要求具备“无记忆”的性质：下一状态的概率分布只能由当前状态决定，在时间序列中它前面的时间均与之无关。这种特定类型的“无记忆性”称作马尔可夫性质。
- 隐马尔可夫模型，是统计模型，它用来描述一个含有隐含未知参数的马尔可夫过程。其难点是从可观察的参数中确定该过程的隐含参数。

## HMM的两个假设
- 齐次马尔可夫假设：第t个隐状态（骰子）只跟前一时刻的t-1隐状态（骰子）有关，与除此之外的其他隐状态（如t-2，t+3）无关。
- 观测独立假设：在任意时刻观测 ot（点数）只依赖于当前时刻的隐状态 it（骰子），与其他时刻的隐状态无关。

## 隐马尔可夫模型数学定义
- 前面的“BIO”的实体标签，是一个不可观测的隐状态，而HMM模型描述的就是由这些隐状态序列（实体标记）生成可观测状态（可读文本）的过程。

设我们的可观测状态序列是由所有汉字组成的集合，我们用 $V_{Observation}$  \,来表示：$V_{obs.}=\{v_1,v_2,...,v_M\}$  
上式中，$v$表示字典中单个字，假设我们已知的字数为$M$。  
设所有可能的隐藏状态集合为$Q_{hidden}$，一共有$N$种隐藏状态，例如我们现在的命名实体识别数据里面只有7种标签：$Q_{hidden}=\{q_1,q_2,...,q_N\}$  
设我们有观测到的一串自然语言序列文本$O$，一共有$T$个字，又有这段观测到的文本多对应的实体标记，也就是隐状态$I$：$I=\{i_1,i_2,...,i_T\}$(隐状态)$O=\{o_1,o_2,...,o_T\}$(观测)
（隐状态）（观测）  
注意上式中，我们常称t为时刻，如上式中一共有$T$个时刻（$T$个汉字）。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594791764375-386983b2-3a72-43d5-88e7-2973e641fbf5.png"/>

## HMM的初始隐状态概率
初始隐状态概率通常用来表示：
                    $π=P(i_1=q_i)$，其中$q_i∈Q_{hidden}=\{q_0,q_1,...,q_{N-1}\}$
序列中第一个观测对象的隐状态是的概率，也就是初始隐状态概率。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594794811592-eed64716-cae1-4dc8-858c-e22f92d11b3b.png"/>

## HMM的转移概率
$P(i_t|i_{t-1})$指的是隐状态i从t-1时刻转向t时刻的概率。用A表示转移概率矩阵（transition matrix），$A_{ij}=P(i_{t+1}=q_j|i_t=q_i)$，其中$q_i\in Q_{hidden}$。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594795446668-8a766613-384c-444f-b291-4482c1e3b33e.png"/>

## HMM的发射概率
- 发射概率$P(o_t|i_t)$描述$o_t$依赖于当前时刻的隐状态$i_t$程度。
用B表示发射概率矩阵（emission matrix）：$B_{jk}=P(o_t=v_k|i_k=q_j)$，其中$q_i\in Q_{hidden}$，$v_k\in V_{obs.}=\{v_0,v_1,...,v_{M-1}\}$

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594795821324-5b9dd45d-f090-465a-b446-af8c5117cab6.png"/>

## 用HMM解决序列标注问题

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594796017056-e8c24529-e077-4258-9721-365b747529f6.png"/>

## HMM的3个基本问题
模型：$(π,A,B)$  
观测序列：$O=(o_1,o_2,...,o_T)$  
状态序列：$I=(i_1,i_2,...,i_T)$  
1. 概率计算问题：已知模型和观测序列，计算在该模型下观测序列出现的概率。即求$\color{teal}{P(O|λ)}$。
2. 学习问题：已知观测序列，估计模型的参数。
学习问题根据训练数据是否包含对应的状态序列，分为：  
    2.1. 监督学习：极大似然估计。即求$\color{teal}{P(λ|O,I)}$。  
    2.2. 非监督学习：EM算法。即求$\color{teal}{P(λ|O)}$。  
3. 预测问题：decoding问题，已知模型参数和观测序列，求最有可能对应的状态序列。即求$\color{teal}{P(I|λ,O)}$。

## 利用极大似然估计进行监督学习
1. 初始状态概率的估计
    - $\hat{π}_{qi}=\frac{count(q^{1}_{i})}{count(o_1)}$ $\sum_iπ_{q_i}=1$
        
2. 转移概率的估计
    - $\hat{A}_{ij}=P(i_{i+1}=q_j|i_t=q_i)=\frac{count(q_iq_j)}{count(q_i)}$   
    - $\sum_jA_{ij}=\sum_jP(i_{t+1}=q_j|i_t=q_i)=1$

3. 发射概率的估计
    - $\hat{B}_{jk}=P(o_t=v_k|i_t=q_j)=\frac{count(q_jv_k)}{count(q_j)}$    
    - $\sum_kB_{jk}=\sum_kP(o_t=v_k|i_t=g_j)=1$
    
## 维特比解码
维特比算法解码使用了动态规划算法来解决HMM的预测问题，找到概率最大路径，也就是最优路径。  
在每一时刻，计算当前时刻落在每种隐状态的最大概率，并记录这个最大概率是从前一时刻哪一隐状态转移过来的，最后再从结尾回溯最大概率，也就是最有可能的最优路径。  
设有N个状态序列长度为T，则时间复杂度为：
1. 穷举法：$O(N^T)$
2. 维特比：$O(TN^2)$

# 3 条件随机场
## 概率图模型
概率图模型是指一种用图结构来描述多元随机变量之间条件独立关系的概率模型。  
图中的每个节点都对应一个随机变量，可以是观察变量、隐变量或是未知参数等；每个连接表示两个随机变量之间具有依赖关系。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594800186545-7937cea0-50f4-401c-9522-eacbe623bd39.png"/>
<img src="https://cdn.nlark.com/yuque/0/2020/jpeg/1508544/1594801843197-d20adcca-e4b8-4f6c-b9ec-041edf0eddbe.jpeg"/>

## 有向图 VS 无向图
- 有向图：联合概率分布可以利用条件概率来表示。$P(v^d_1,...,v^d_n)=\prod_{i=1}^nP(v_i^d|v^d_{πi})$
- 无向图：通过因子分解将无向图所描述的联合概率分布表达为若干个子联合概率的乘积。$P(Y)=\frac{1}{Z}\prod_C\psi_C(Y_C)$

## 因子分解
- 无向图G中任何两个结点均有边连接的节点子集称为团。若C是无向图G的一个团，并且不能再加进任何一个G的结点使其成为一个更大的团，则称此C为最大团。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594802494622-b5d6323c-a8bb-4b44-85f5-586e5b530567.png"/>

## Hammersly-Clifford定理
- 无向图的联合概率可以分解为一系列定义在最大团上的非负函数的乘积形式。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594802744460-e834f9b8-0f1a-4580-a11b-7a77c19147f2.png"/>

## 条件随机场（Conditional random field, CRF）
- 定义：如果随机变量Y构成一个由无向图$G=(V,E)$表示的马尔可夫随机场，对任意节点$v\in V$都成立，即$P(Y_v|X,Y_w,w\neq v)=P(Y_v|X,Y_w,w\sim v)$
- 则称$P(Y|X)$是条件随机场。式中$w\neq v$表示w是除v以外的所有节点，$w\sim v$表示w是与v相连接的所有节点。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594803274901-ad100126-ea60-4314-b9fa-c54c5c9e1ae7.png"/>

- CRF是条件概率分布模型$P(Y|X)$，表示的是给定一组输入随机变量X的条件下另一组输出随机变量Y的马尔可夫随机场，也就是说CRF的特点是假设输出随机变量构成马尔可夫随机场。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594812709431-f5cb34ee-99e7-4d06-8e70-3c66de527802.png"/>
<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594812722843-49edd32d-05e8-4709-a848-96cd5616513e.png"/>

- 这里说的CRF指的是用于序列标注问题的线性链条件随机场，是由输入序列来预测输出序列的判别式模型。

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594812872344-ebe47943-902f-4b21-94a1-e2d06b158619.png"/>
<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594812887514-bc66b3da-322e-4997-be16-a4dae8630020.png"/>

## 生成 VS 判别
- 生成：对联合概率$P(X,Y)$建模；判别：对条件概率$P(Y|X)$建模
- $P(Y|X)=\frac{P(X,Y)}{P(X)}$

<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594813091361-69343bf0-d3cc-4758-afe6-019739f4b78b.png"/>

## 线性链条件随机场
无向图的每条边都存在于状态序列Y的相邻两个节点，最大团C是相邻两个节点的集合，X和Y有相同的图结构意味着每个$X_i$都与$Y_i$一一对应。  
设两组随机变量$X=(X_1,...,X_n)$，$Y=(Y_1,...,Y_n)$，那么线性链条件随机场的定义为：$P(Y_i|X,Y_1,...,Y_{i-1},Y_{i+1},...,Y_n)=P(Y_i|X,Y_{i-1},Y_{i+1}),i=1,...,n$  
其中，当取1或n时只考虑单边。

## 参数化形式
- 给定一个线性链条件随机场$P(Y|X)$，当观测序列为$x=x_1x_2...$时，状态序列为$y=y_1y_2...$的概率为（实际上应该写为$P(Y=y|x;\theta)$）  
    - $\color{blue}{P(Y=y|x)=\frac{1}{Z(x)}exp(\sum_{k}\lambda_{k}\sum_{i}t_k(y_{i-1},y_i,x,i)+\sum_l\mu_l\sum_is_l(y_i,x,i))}$  
    - $Z(x)=\sum_yexp(\sum_k\lambda_k\sum_it_k(y_{i-1},y_i,x,i)+\sum_l\mu_l\sum_is_l(y_i,x,i))$  
- $Z(x)$作为规范化因子，是对y的所有可能取值求和。

## 特征函数
- 转移特征$t_k(y_{i-1},y_i,x,i)$是定义在边上的特征函数（transition），依赖于当前位置i和前一位置i-1，对应的权值为$\lambda_k$。
- 状态特征$s_l(y_i,x,i)$是定义在节点上的特征函数（state），依赖于当前位置i，对应的权值为$\mu_l$。
- 特征函数的取值为1或0，当满足规定好的特征条件时取值为1，否则为0。

## 特征函数例子
1. $\color{teal}{s_1(y_i,x,i)}$  
$s_1=
\begin{cases}
1, &\text{if $y_i$ is E-SYM, $x_i$ is '痛'}\\
0, &\text{otherwise}
\end{cases}$  
权重大，就表明倾向于将痛标注为症状的结尾。  
2. $\color{teal}{s_2(y_i,x,i)}$  
$s_2=
\begin{cases}
1, &\text{if $y_i$ is E-TEST, $x_{i+1}$ is ':'}\\
0, &\text{otherwise}
\end{cases}$  
正权重：倾向于将冒号前的字标注为检验的结尾，例如'舒张压: 80'。
3. $\color{teal}{t_3(y_{i-1},y_i,x,i)}$  
$t_3=
\begin{cases}
1, &\text{if $y_{i-1}$ is B-SYM, $y_{i}$ is I-SYM}\\
0, &\text{otherwise}
\end{cases}$  
权重大，就表明倾向于认为B-SYM后面跟着I-SYM。  
4. $\color{teal}{t_4(y_{i-1},y_i,x,i)}$  
$t_4=
\begin{cases}
1, &\text{if $y_{i-1}$ is B-SYM, $y_{i}$ is E-DIS}\\
0, &\text{otherwise}
\end{cases}$
权重小，就表明倾向于认为B-SYM后面不会跟着E-DIS。

## 从特征到概率
实际上CRF就是序列版本的逻辑回归（logistic regression）。正如逻辑回归是分类问题的对数线性模型，CRF是序列标注问题的对数线性模型。
$score(l|s)=\sum_{j=1}^{m}\sum_{i=1}^{n}\lambda_j f_j(s,i,l_i,l_{i-1})$（第一个求和是对遍历特征方程j的求和，而内层的求和是对句子里面的每一个位置i进行遍历进行求和）
最终，我们通过求指数与归一化的方式将这些得分转换为0、1之间的概率值：
$p(l|s)=\frac{exp[score(l|s)]}{\sum_{l^{'}}exp[score(l^{'}|s)]}
=\frac{exp[\sum_{j=1}^{m}\sum_{i=1}^{n}\lambda_jf_j(s,i,l_i,l_{i-1})]}{\sum_{l^{'}}exp[\sum_{j=1}^{m}\sum_{i=1}^{n}\lambda_jf_j(s,i,l_i^{'},l_{i-1}^{'})]}$

## CRF VS HMM
CRF更加强大      
• CRF可以为任何HMM能够建模的事物建模，甚至更多。  
• CRF可以定义更加广泛的特征集。HMM在本质上必然是局部的，而CRF就可以使用更加全局的特征。  
• CRF可以有任意权重值，HMM的概率值必须满足特定的约束。  
$p(l|s)=p(l_1)\prod_ip(l_i|l_{i-1})p(w_i|l_i)$   
$logp(l,s)=logp(l_1)+\sum_ilogp(l_i|l_{i-1})+\sum_ilogp(w_i|l_i)$  
• 为每个HMM的转换概率$p(l_i=y|l_{i-1}=x)$，定义一组CRF转换特征如$f_{x,y}(s,i,l_i,l_{i-1})=1$ if $l_i=y$且$l_{i-1}=x$的。给定每个特征权重值为$w_{x.y}=logp(l_i=y|l_{i-1}=x)$。  
• 类似的，为每个HMM的发射概率$p(w_i=z|l_i=x)$，定义一组CRF发射特征如$g_{x,y}(s,i,l_i,l_{i-1})$ if $w_i=z$且$l_i=x$。给定每个特征的权重值为$w_{x,z}=logp(w_i=z|l_i=x)$。  

## CRF总结
<img src="https://cdn.nlark.com/yuque/0/2020/png/1508544/1594883256410-8bf84b25-a749-458c-9353-0252d35e2f2d.png"/>

- “万创杯”中医药天池大数据竞赛——[中药说明书实体识别挑战](https://tianchi.aliyun.com/competition/entrance/531824/introduction)
# 数据说明
- 本次标注数据源来自中药药品说明书，共包含1997份去重后的药品说明书，其中1000份用于训练数据，500份用作初赛测试数据，剩余的497份用作复赛的测试数据。本次复赛测试数据不对外开放，不可下载且不可见，选手需要在天池平台通过镜像方式提交。共定义了13类实体，具体类别定义如下：
    - **药品(DRUG)**:中药名称，指在中医理论指导下，用于预防、治疗、诊断疾病并具有康复与保健作用的物质。中药主要来源于天然药及其加工品，包括植物药、动物药、矿物药及部分化学、生物制品类药物。例子: 六味地黄丸、逍遥散
    - **药物成分(DRUG_INGREDIENT)**: 中药组成成分，指中药复方中所含有的所有与该复方临床应用目的密切相关的药理活性成分。例子:当归、人参、枸杞
    - **疾病(DISEASE)**: 疾病名称，指人体在一定原因的损害性作用下，因自稳调节紊乱而发生的异常生命活动过程，是特定的异常病理情形，而且会影响生物体的部分或是所有器官。通常解释为“身体病况”（medical condition），而且伴随着特定的症状及医学征象。例子：高血压、心绞痛、糖尿病
    - **症状(SYMPTOM)**: 指疾病过程中机体内的一系列机能、代谢和形态结构异常变化所引起的病人主观上的异常感觉或某些客观病态改变。例子_：头晕、心悸、小腹胀痛_
    - **证候(SYNDROME)**: 中医学专用术语，概括为一系列有相互关联的症状总称，即通过望、闻、问、切四诊所获知的疾病过程中表现在整体层次上的机体反应状态及其运动、变化，简称证或者候，是指不同症状和体征的综合表现，单一的症状和体征无法表现一个完整的证候。 例子：血瘀、气滞、气血不足、气血两虚
    - **疾病分组(DISEASE_GROUP)**: 疾病涉及有人体组织部位的疾病名称的统称概念，非某项具体医学疾病。例子：肾病、肝病、肺病
    - **食物(FOOD)**:指能够满足机体正常生理和生化能量需求，并能延续正常寿命的物质。对人体而言，能够满足人的正常生活活动需求并利于寿命延长的物质称之为食物。例子：苹果、茶、木耳、萝卜
    - **食物分组(FOOD_GROUP)**: 中医中饮食养生中，将食物分为寒热温凉四性，同时中医药禁忌中对于具有某类共同属性食物的统称，记为食物分组。例子：油腻食物、辛辣食物、凉性食物
    - **人群(PERSON_GROUP)**: 中医药的适用及禁忌范围内相关特定人群。例子：孕妇、经期妇女、儿童、青春期少女
    - **药品分组(DRUG_GROUP)**: 具有某一类共同属性的药品类统称概念，非某项具体药品名。例子：止咳药、退烧药
    - **药物剂型(DRUG_DOSAGE)**: 药物在供给临床使用前，均必须制成适合于医疗和预防应用的形式，成为药物剂型。例子：浓缩丸、水蜜丸、糖衣片
    - **药物性味(DRUG_TASTE)**: 药品的性质和气味。例子：味甘、酸涩、气凉
    - **中药功效(DRUG_EFFICACY)**: 药品的主治功能和效果的统称，例子：滋阴补肾、去瘀生新、活血化瘀

# 评估标准
- 本次挑战是标准的NER任务，以strict-F1作为衡量标准。

# 评测指标
- 本任务采用严格F1-Measure作为评测指标。$S=\{S_1, S_1, ...,S_m\}$，人工标注的结果（Gold Standard）集合记为$G=\{g_1, g_2, ..., g_n\}$。集合元素为一个实体提及，表示为四元组$<d,pos_b,pos_e,c>$，d表示文档，$pos_b$和$pos_e$分别对应实体提及在文档d中的起止下标，c表示实体提及所属预定义类别。评测以Micro F1值作为最终排名依据。

# 严格指标（Strict）
- 我们定义$s_i\in S$与$g_j\in G$严格等价，当且仅当：
    - $s_i · d=g_j·d$
    - $s_i·pos_b=g_j·pos_b$
    - $s_i·pos_e=g_j·pos_e$
    - $s_i·c=g_j·c$
- 基于以上等价关系，我们定义集合S与G的严格交集为$P=\frac{|S\cap G|}{|S|}$，$R=\frac{|S \cap G|}{|G|}$。
- 由此得到严格评测指标为：$F_1=\frac{2PR}{P+R}$

# 练习题
参考[Github](https://github.com/lonePatient/BERT-NER-Pytorch)使用BERT+CRF实现对中药说明书实体识别挑战的中文命名实体识别。